In [6]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_API_KEY")

In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [3]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [4]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [44]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

wbl = WebBaseLoader("https://docs.smith.langchain.com/")
documents = wbl.load()
split_docs = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(documents)
hf_embed = OpenAIEmbeddings() # HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db_faiss = FAISS.from_documents(split_docs, hf_embed)
retriever = db_faiss.as_retriever()


In [45]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "LangSmith-Search", "Search any info in Lang Smith")



In [50]:
from langchain_groq import ChatGroq
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [wiki_tool, arxiv_tool, retriever_tool]

llm = ChatGroq(model="llama3-8b-8192")
agent = create_openai_tools_agent(llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)


In [38]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [52]:
try:
    response = agent_executor.invoke({"input": "What is Deep Learning?"})
    print(response)
except Exception as e:
    print(e)
    if hasattr(e,'failed_generation'):
        print(e.failed_generation)


{'input': 'What is Deep Learning?', 'output': "Based on the provided information, I can provide a direct answer:\n\nDeep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. It takes inspiration from biological neural networks and is capable of learning and improving on its own.\n\nMachine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without being explicitly programmed.\n\nNeural networks are a group of interconnected units called neurons that send signals to one another, and can be either biological cells or mathematical models. They can be simple individually, but many of them together can process complex patterns and learn from data.\n\nHere's a summary:\n\nDeep learning is a subset of machine learning that uses neural